# Large Language Models (LLM) Overview

This notebook provides a comprehensive overview of Large Language Models, covering fundamental concepts, architectures, and training methodologies.

## Table of Contents
1. [LLM Fundamentals](#1-llm-fundamentals)
2. [Transformer Architecture](#2-transformer-architecture)
3. [Attention Mechanism](#3-attention-mechanism)
4. [Tokenization](#4-tokenization)
5. [Key Models](#5-key-models)
6. [Training Stages](#6-training-stages)
7. [Scaling Laws](#7-scaling-laws)

---
## 1. LLM Fundamentals

### What is a Large Language Model?

A **Large Language Model (LLM)** is a neural network trained on massive amounts of text data to understand and generate human-like text. These models learn statistical patterns in language and can perform various tasks:

- **Text generation**: Writing essays, code, stories
- **Question answering**: Providing informative responses
- **Summarization**: Condensing long documents
- **Translation**: Converting between languages
- **Reasoning**: Solving logical and mathematical problems

### Core Concept: Next Token Prediction

```
┌─────────────────────────────────────────────────────────────┐
│                  Next Token Prediction                       │
├─────────────────────────────────────────────────────────────┤
│                                                              │
│  Input:  "The cat sat on the"                               │
│                    │                                         │
│                    ▼                                         │
│            ┌─────────────┐                                   │
│            │    LLM      │                                   │
│            └─────────────┘                                   │
│                    │                                         │
│                    ▼                                         │
│  Probability Distribution:                                   │
│    "mat"   → 0.45                                           │
│    "floor" → 0.25                                           │
│    "bed"   → 0.15                                           │
│    "roof"  → 0.08                                           │
│    ...     → 0.07                                           │
│                                                              │
└─────────────────────────────────────────────────────────────┘
```

LLMs are trained using **autoregressive language modeling**: predicting the next token given all previous tokens.

In [ ]:
# Simplified illustration of next-token prediction concept
import numpy as np

def softmax(logits):
    """Convert logits to probabilities"""
    exp_logits = np.exp(logits - np.max(logits))
    return exp_logits / exp_logits.sum()

# Simulated vocabulary and logits from an LLM
vocabulary = ["mat", "floor", "bed", "roof", "table", "chair"]
logits = np.array([2.5, 1.8, 1.2, 0.5, 0.3, 0.1])  # Raw model outputs

probabilities = softmax(logits)

print("Input: 'The cat sat on the ___'\n")
print("Token Probabilities:")
for token, prob in sorted(zip(vocabulary, probabilities), key=lambda x: -x[1]):
    bar = "█" * int(prob * 40)
    print(f"  {token:8} {prob:.3f} {bar}")

### Key Characteristics of LLMs

| Characteristic | Description |
|---------------|-------------|
| **Scale** | Billions to trillions of parameters |
| **Training Data** | Terabytes of text from internet, books, code |
| **Architecture** | Transformer-based (usually decoder-only) |
| **Emergent Abilities** | Capabilities that appear at larger scales |
| **In-context Learning** | Learning from examples in the prompt |
| **Zero-shot Capability** | Performing tasks without task-specific training |

---
## 2. Transformer Architecture

The **Transformer** architecture, introduced in "Attention Is All You Need" (Vaswani et al., 2017), is the foundation of modern LLMs.

### High-Level Architecture

```
┌────────────────────────────────────────────────────────────────────┐
│                    Transformer Architecture                         │
├────────────────────────────────────────────────────────────────────┤
│                                                                     │
│  ┌──────────────────────┐     ┌──────────────────────┐             │
│  │      ENCODER         │     │      DECODER         │             │
│  │  (e.g., BERT)        │     │  (e.g., GPT)         │             │
│  ├──────────────────────┤     ├──────────────────────┤             │
│  │                      │     │                      │             │
│  │  ┌────────────────┐  │     │  ┌────────────────┐  │             │
│  │  │ Self-Attention │  │     │  │Masked Self-Attn│  │             │
│  │  └───────┬────────┘  │     │  └───────┬────────┘  │             │
│  │          │           │     │          │           │             │
│  │  ┌───────▼────────┐  │     │  ┌───────▼────────┐  │             │
│  │  │   Add & Norm   │  │     │  │   Add & Norm   │  │             │
│  │  └───────┬────────┘  │     │  └───────┬────────┘  │             │
│  │          │           │     │          │           │             │
│  │  ┌───────▼────────┐  │     │  ┌───────▼────────┐  │             │
│  │  │  Feed Forward  │  │     │  │  Feed Forward  │  │             │
│  │  └───────┬────────┘  │     │  └───────┬────────┘  │             │
│  │          │           │     │          │           │             │
│  │  ┌───────▼────────┐  │     │  ┌───────▼────────┐  │             │
│  │  │   Add & Norm   │  │     │  │   Add & Norm   │  │             │
│  │  └────────────────┘  │     │  └────────────────┘  │             │
│  │                      │     │                      │             │
│  │     × N layers       │     │     × N layers       │             │
│  └──────────────────────┘     └──────────────────────┘             │
│                                                                     │
│  Encoder: Bidirectional        Decoder: Autoregressive             │
│  (sees all tokens)             (sees only past tokens)             │
│                                                                     │
└────────────────────────────────────────────────────────────────────┘
```

### Types of Transformer Models

| Type | Examples | Use Case |
|------|----------|----------|
| **Encoder-only** | BERT, RoBERTa | Classification, NER, embeddings |
| **Decoder-only** | GPT, Claude, LLaMA | Text generation, chatbots |
| **Encoder-Decoder** | T5, BART | Translation, summarization |

### Decoder Block Details (GPT-style)

```
┌─────────────────────────────────────────────────────────────────┐
│                    Single Decoder Block                          │
├─────────────────────────────────────────────────────────────────┤
│                                                                  │
│                      Input (x)                                   │
│                         │                                        │
│                         ▼                                        │
│                  ┌──────────────┐                                │
│                  │  Layer Norm  │                                │
│                  └──────┬───────┘                                │
│                         │                                        │
│                         ▼                                        │
│              ┌──────────────────────┐                            │
│              │  Multi-Head Causal   │                            │
│              │  Self-Attention      │                            │
│              └──────────┬───────────┘                            │
│                         │                                        │
│         x ─────────────(+)◄──────────  (Residual Connection)     │
│                         │                                        │
│                         ▼                                        │
│                  ┌──────────────┐                                │
│                  │  Layer Norm  │                                │
│                  └──────┬───────┘                                │
│                         │                                        │
│                         ▼                                        │
│              ┌──────────────────────┐                            │
│              │   Feed-Forward Net   │                            │
│              │   (MLP: up → down)   │                            │
│              └──────────┬───────────┘                            │
│                         │                                        │
│         x ─────────────(+)◄──────────  (Residual Connection)     │
│                         │                                        │
│                         ▼                                        │
│                    Output (y)                                    │
│                                                                  │
└─────────────────────────────────────────────────────────────────┘
```

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import math

class TransformerBlock(nn.Module):
    """A single transformer decoder block (GPT-style, pre-norm)"""
    
    def __init__(self, d_model: int, n_heads: int, d_ff: int, dropout: float = 0.1):
        super().__init__()
        self.ln1 = nn.LayerNorm(d_model)
        self.attn = nn.MultiheadAttention(d_model, n_heads, dropout=dropout, batch_first=True)
        self.ln2 = nn.LayerNorm(d_model)
        self.ffn = nn.Sequential(
            nn.Linear(d_model, d_ff),
            nn.GELU(),
            nn.Linear(d_ff, d_model),
            nn.Dropout(dropout)
        )
        self.dropout = nn.Dropout(dropout)
    
    def forward(self, x: torch.Tensor, mask: torch.Tensor = None) -> torch.Tensor:
        # Pre-norm architecture (used in GPT-2, LLaMA)
        # Self-attention with residual
        normed = self.ln1(x)
        attn_out, _ = self.attn(normed, normed, normed, attn_mask=mask, is_causal=True)
        x = x + self.dropout(attn_out)
        
        # FFN with residual
        normed = self.ln2(x)
        x = x + self.ffn(normed)
        
        return x

# Example usage
d_model = 512   # Embedding dimension
n_heads = 8     # Number of attention heads
d_ff = 2048     # FFN hidden dimension (typically 4x d_model)

block = TransformerBlock(d_model, n_heads, d_ff)
x = torch.randn(2, 10, d_model)  # (batch, seq_len, d_model)
output = block(x)

print(f"Input shape:  {x.shape}")
print(f"Output shape: {output.shape}")
print(f"\nBlock parameters: {sum(p.numel() for p in block.parameters()):,}")

---
## 3. Attention Mechanism

The **attention mechanism** is the core innovation of transformers. It allows each token to "attend" to all other tokens and gather relevant information.

### Scaled Dot-Product Attention

$$\text{Attention}(Q, K, V) = \text{softmax}\left(\frac{QK^T}{\sqrt{d_k}}\right)V$$

Where:
- **Q (Query)**: What am I looking for?
- **K (Key)**: What do I contain?
- **V (Value)**: What information should I pass?
- **$d_k$**: Dimension of keys (for scaling)

```
┌─────────────────────────────────────────────────────────────────────┐
│                    Scaled Dot-Product Attention                      │
├─────────────────────────────────────────────────────────────────────┤
│                                                                      │
│     Q          K          V                                          │
│     │          │          │                                          │
│     │          │          │                                          │
│     └────┬─────┘          │                                          │
│          │                │                                          │
│          ▼                │                                          │
│    ┌──────────┐           │                                          │
│    │  MatMul  │  (Q @ K.T)│                                          │
│    └────┬─────┘           │                                          │
│         │                 │                                          │
│         ▼                 │                                          │
│    ┌──────────┐           │                                          │
│    │  Scale   │  (÷ √dk) │                                          │
│    └────┬─────┘           │                                          │
│         │                 │                                          │
│         ▼                 │                                          │
│    ┌──────────┐           │                                          │
│    │   Mask   │ (optional)│                                          │
│    └────┬─────┘           │                                          │
│         │                 │                                          │
│         ▼                 │                                          │
│    ┌──────────┐           │                                          │
│    │ Softmax  │           │                                          │
│    └────┬─────┘           │                                          │
│         │                 │                                          │
│         └────────┬────────┘                                          │
│                  │                                                   │
│                  ▼                                                   │
│            ┌──────────┐                                              │
│            │  MatMul  │  (attn_weights @ V)                          │
│            └────┬─────┘                                              │
│                 │                                                    │
│                 ▼                                                    │
│              Output                                                  │
│                                                                      │
└─────────────────────────────────────────────────────────────────────┘
```

In [ ]:
def scaled_dot_product_attention(Q, K, V, mask=None):
    """
    Compute scaled dot-product attention.
    
    Args:
        Q: Query tensor (batch, seq_len, d_k)
        K: Key tensor (batch, seq_len, d_k)
        V: Value tensor (batch, seq_len, d_v)
        mask: Optional attention mask
    
    Returns:
        output: Weighted sum of values
        attention_weights: Attention probabilities
    """
    d_k = Q.size(-1)
    
    # Compute attention scores
    scores = torch.matmul(Q, K.transpose(-2, -1)) / math.sqrt(d_k)
    
    # Apply mask (for causal/decoder attention)
    if mask is not None:
        scores = scores.masked_fill(mask == 0, float('-inf'))
    
    # Softmax to get attention weights
    attention_weights = F.softmax(scores, dim=-1)
    
    # Weighted sum of values
    output = torch.matmul(attention_weights, V)
    
    return output, attention_weights

# Example: Simple attention computation
batch_size, seq_len, d_k = 1, 4, 8
Q = torch.randn(batch_size, seq_len, d_k)
K = torch.randn(batch_size, seq_len, d_k)
V = torch.randn(batch_size, seq_len, d_k)

# Create causal mask (lower triangular)
causal_mask = torch.tril(torch.ones(seq_len, seq_len))

output, attn_weights = scaled_dot_product_attention(Q, K, V, causal_mask)

print("Causal Attention Mask:")
print(causal_mask.numpy())
print("\nAttention Weights (after masking):")
print(attn_weights.squeeze().detach().numpy().round(3))

### Multi-Head Attention

Instead of performing single attention, we use **multiple attention heads** in parallel, allowing the model to attend to information from different representation subspaces.

```
┌──────────────────────────────────────────────────────────────────────┐
│                       Multi-Head Attention                            │
├──────────────────────────────────────────────────────────────────────┤
│                                                                       │
│              Input (d_model = 512)                                    │
│                      │                                                │
│    ┌─────────────────┼─────────────────┐                              │
│    │                 │                 │                              │
│    ▼                 ▼                 ▼                              │
│  ┌────┐           ┌────┐           ┌────┐                             │
│  │ Wq │           │ Wk │           │ Wv │   Linear projections        │
│  └─┬──┘           └─┬──┘           └─┬──┘                             │
│    │                │                │                                │
│    ▼                ▼                ▼                                │
│  ┌───────────────────────────────────────┐                            │
│  │     Split into h=8 heads              │                            │
│  │     (d_k = d_model/h = 64 per head)   │                            │
│  └───────────────────────────────────────┘                            │
│    │                                                                  │
│    ├──► Head 1: Attention(Q₁, K₁, V₁)                                │
│    ├──► Head 2: Attention(Q₂, K₂, V₂)                                │
│    ├──► Head 3: Attention(Q₃, K₃, V₃)                                │
│    │     ...                                                          │
│    └──► Head 8: Attention(Q₈, K₈, V₈)                                │
│                      │                                                │
│                      ▼                                                │
│            ┌─────────────────┐                                        │
│            │   Concatenate   │                                        │
│            └────────┬────────┘                                        │
│                     │                                                 │
│                     ▼                                                 │
│               ┌──────────┐                                            │
│               │    Wo    │  Output projection                         │
│               └────┬─────┘                                            │
│                    │                                                  │
│                    ▼                                                  │
│              Output (d_model)                                         │
│                                                                       │
└──────────────────────────────────────────────────────────────────────┘
```

**Why Multi-Head?**
- Each head can learn different types of relationships
- Head 1 might focus on syntactic relationships
- Head 2 might capture semantic similarity
- Head 3 might track positional patterns

In [ ]:
class MultiHeadAttention(nn.Module):
    """Multi-Head Attention implementation from scratch"""
    
    def __init__(self, d_model: int, n_heads: int, dropout: float = 0.1):
        super().__init__()
        assert d_model % n_heads == 0, "d_model must be divisible by n_heads"
        
        self.d_model = d_model
        self.n_heads = n_heads
        self.d_k = d_model // n_heads
        
        # Linear projections for Q, K, V
        self.W_q = nn.Linear(d_model, d_model)
        self.W_k = nn.Linear(d_model, d_model)
        self.W_v = nn.Linear(d_model, d_model)
        self.W_o = nn.Linear(d_model, d_model)
        
        self.dropout = nn.Dropout(dropout)
    
    def forward(self, x: torch.Tensor, mask: torch.Tensor = None) -> torch.Tensor:
        batch_size, seq_len, _ = x.size()
        
        # Project and reshape: (batch, seq, d_model) -> (batch, n_heads, seq, d_k)
        Q = self.W_q(x).view(batch_size, seq_len, self.n_heads, self.d_k).transpose(1, 2)
        K = self.W_k(x).view(batch_size, seq_len, self.n_heads, self.d_k).transpose(1, 2)
        V = self.W_v(x).view(batch_size, seq_len, self.n_heads, self.d_k).transpose(1, 2)
        
        # Compute attention
        scores = torch.matmul(Q, K.transpose(-2, -1)) / math.sqrt(self.d_k)
        
        if mask is not None:
            scores = scores.masked_fill(mask == 0, float('-inf'))
        
        attn_weights = F.softmax(scores, dim=-1)
        attn_weights = self.dropout(attn_weights)
        
        # Apply attention to values
        context = torch.matmul(attn_weights, V)
        
        # Concatenate heads and project
        context = context.transpose(1, 2).contiguous().view(batch_size, seq_len, self.d_model)
        output = self.W_o(context)
        
        return output

# Demonstrate
mha = MultiHeadAttention(d_model=256, n_heads=8)
x = torch.randn(2, 16, 256)  # (batch, seq_len, d_model)
output = mha(x)

print(f"Multi-Head Attention")
print(f"  d_model: 256, n_heads: 8, d_k: 32")
print(f"  Input shape:  {x.shape}")
print(f"  Output shape: {output.shape}")
print(f"  Parameters: {sum(p.numel() for p in mha.parameters()):,}")

---
## 4. Tokenization

**Tokenization** is the process of converting raw text into a sequence of discrete tokens that the model can process.

### Tokenization Methods

```
┌─────────────────────────────────────────────────────────────────────┐
│                     Tokenization Methods                             │
├─────────────────────────────────────────────────────────────────────┤
│                                                                      │
│  Input: "The transformer architecture is amazing!"                  │
│                                                                      │
│  ┌─────────────────────────────────────────────────────────────┐    │
│  │ Word-level:                                                  │    │
│  │ ["The", "transformer", "architecture", "is", "amazing", "!"] │    │
│  │ ⚠️ Large vocabulary, OOV problems                            │    │
│  └─────────────────────────────────────────────────────────────┘    │
│                                                                      │
│  ┌─────────────────────────────────────────────────────────────┐    │
│  │ Character-level:                                             │    │
│  │ ["T","h","e"," ","t","r","a","n","s","f","o","r","m",...]   │    │
│  │ ⚠️ Very long sequences, hard to learn semantics              │    │
│  └─────────────────────────────────────────────────────────────┘    │
│                                                                      │
│  ┌─────────────────────────────────────────────────────────────┐    │
│  │ Subword (BPE/WordPiece/Unigram):                    ✓ Best   │    │
│  │ ["The", " transform", "er", " architecture", " is", ...]   │    │
│  │ ✓ Balanced vocabulary, handles rare words                    │    │
│  └─────────────────────────────────────────────────────────────┘    │
│                                                                      │
└─────────────────────────────────────────────────────────────────────┘
```

### Popular Tokenizers

| Tokenizer | Used By | Vocab Size | Key Features |
|-----------|---------|------------|---------------|
| **BPE** | GPT-2, GPT-3 | 50,257 | Byte-level, no UNK tokens |
| **SentencePiece** | LLaMA, T5 | ~32,000 | Language-agnostic |
| **WordPiece** | BERT | 30,522 | Subword with ## prefix |
| **tiktoken** | GPT-4, Claude | ~100,000 | Fast, efficient |

In [ ]:
# Demonstrate tokenization with tiktoken (OpenAI's tokenizer)
try:
    import tiktoken
    
    # GPT-4 uses cl100k_base encoding
    enc = tiktoken.get_encoding("cl100k_base")
    
    text = "The transformer architecture revolutionized natural language processing!"
    
    # Encode
    tokens = enc.encode(text)
    
    print(f"Original text: '{text}'")
    print(f"\nNumber of tokens: {len(tokens)}")
    print(f"Token IDs: {tokens}")
    print(f"\nToken breakdown:")
    for token_id in tokens:
        token_text = enc.decode([token_id])
        print(f"  {token_id:6d} → '{token_text}'")
    
    # Decode back
    decoded = enc.decode(tokens)
    print(f"\nDecoded text: '{decoded}'")
    print(f"Round-trip successful: {text == decoded}")
    
except ImportError:
    print("tiktoken not installed. Install with: pip install tiktoken")
    print("\nSimulated tokenization example:")
    print("  'Hello world' → [15496, 995]")
    print("  'Transformer'  → [49688, Former]")

In [ ]:
# Simple BPE implementation for educational purposes
from collections import Counter, defaultdict
import re

def get_vocab(text: str) -> dict:
    """Initialize vocabulary with character-level tokens"""
    words = text.lower().split()
    vocab = Counter()
    for word in words:
        # Add end-of-word marker
        word_tokens = tuple(list(word) + ['</w>'])
        vocab[word_tokens] += 1
    return vocab

def get_pairs(vocab: dict) -> Counter:
    """Get frequency of adjacent pairs"""
    pairs = Counter()
    for word, freq in vocab.items():
        for i in range(len(word) - 1):
            pairs[(word[i], word[i+1])] += freq
    return pairs

def merge_vocab(pair: tuple, vocab: dict) -> dict:
    """Merge most frequent pair in vocabulary"""
    new_vocab = {}
    bigram = pair
    replacement = ''.join(pair)
    
    for word, freq in vocab.items():
        new_word = []
        i = 0
        while i < len(word):
            if i < len(word) - 1 and (word[i], word[i+1]) == bigram:
                new_word.append(replacement)
                i += 2
            else:
                new_word.append(word[i])
                i += 1
        new_vocab[tuple(new_word)] = freq
    
    return new_vocab

# Demonstrate BPE on sample text
text = "the cat sat on the mat the cat is fat"
vocab = get_vocab(text)

print("BPE (Byte Pair Encoding) Demo")
print("="*50)
print(f"\nInitial text: '{text}'")
print(f"\nInitial vocabulary (character-level):")
for word, freq in sorted(vocab.items(), key=lambda x: -x[1])[:5]:
    print(f"  {' '.join(word):20} freq={freq}")

# Perform a few BPE merges
num_merges = 5
print(f"\nPerforming {num_merges} BPE merges:")
for i in range(num_merges):
    pairs = get_pairs(vocab)
    if not pairs:
        break
    best_pair = pairs.most_common(1)[0][0]
    vocab = merge_vocab(best_pair, vocab)
    print(f"  Merge {i+1}: '{best_pair[0]}' + '{best_pair[1]}' → '{best_pair[0]+best_pair[1]}'")

print(f"\nFinal vocabulary after merges:")
for word, freq in sorted(vocab.items(), key=lambda x: -x[1])[:5]:
    print(f"  {' '.join(word):20} freq={freq}")

---
## 5. Key Models

### Evolution of LLMs

```
┌────────────────────────────────────────────────────────────────────────────┐
│                        LLM Evolution Timeline                               │
├────────────────────────────────────────────────────────────────────────────┤
│                                                                             │
│  2017    2018      2019       2020        2022        2023       2024       │
│    │       │         │          │           │           │          │        │
│    ▼       ▼         ▼          ▼           ▼           ▼          ▼        │
│  Trans-  BERT     GPT-2      GPT-3      ChatGPT     GPT-4     Claude 3    │
│  former  110M     1.5B       175B       (RLHF)      ~1.7T?    LLaMA 3      │
│                                                     Claude 2              │
│    │       │         │          │           │           │          │        │
│    └───────┴─────────┴──────────┴───────────┴───────────┴──────────┘        │
│                                                                             │
│  Key Milestones:                                                            │
│  • 2017: Attention Is All You Need (Transformer)                            │
│  • 2018: BERT - Bidirectional pretraining                                   │
│  • 2019: GPT-2 - "Too dangerous to release"                                 │
│  • 2020: GPT-3 - In-context learning, few-shot                              │
│  • 2022: ChatGPT - RLHF, conversational AI goes mainstream                  │
│  • 2023: GPT-4/Claude 2 - Multimodal, improved reasoning                    │
│  • 2024: Claude 3/LLaMA 3 - Open weights, competition intensifies           │
│                                                                             │
└────────────────────────────────────────────────────────────────────────────┘
```

### GPT Family (OpenAI)

**GPT (Generative Pre-trained Transformer)** models are decoder-only transformers trained for autoregressive language modeling.

| Model | Parameters | Context | Key Features |
|-------|------------|---------|---------------|
| GPT-2 | 1.5B | 1,024 | Open weights, trained on WebText |
| GPT-3 | 175B | 2,048 | Few-shot learning, API-only |
| GPT-3.5 | ~20B? | 4,096 | RLHF, ChatGPT |
| GPT-4 | ~1.7T? (MoE) | 8K-128K | Multimodal, improved reasoning |
| GPT-4o | Unknown | 128K | Fast, multimodal native |

### Claude Family (Anthropic)

**Claude** models are developed by Anthropic with a focus on safety, helpfulness, and honesty.

| Model | Context | Key Features |
|-------|---------|---------------|
| Claude 1 | 9K | Constitutional AI |
| Claude 2 | 100K | Longer context, improved coding |
| Claude 3 Haiku | 200K | Fast, efficient |
| Claude 3 Sonnet | 200K | Balanced speed/capability |
| Claude 3 Opus | 200K | Most capable |
| Claude 3.5 Sonnet | 200K | Improved reasoning, artifacts |

**Key Innovations:**
- **Constitutional AI (CAI)**: Self-improvement through principles
- **RLHF with human feedback**: Alignment with human values
- **Harmlessness training**: Reducing harmful outputs

### LLaMA Family (Meta)

**LLaMA (Large Language Model Meta AI)** provides open-weight models for research.

| Model | Parameters | Context | Key Features |
|-------|------------|---------|---------------|
| LLaMA 1 | 7B-65B | 2K | Open weights for research |
| LLaMA 2 | 7B-70B | 4K | Commercial license, chat-tuned |
| LLaMA 3 | 8B-70B | 8K | Improved training, tokenizer |
| LLaMA 3.1 | 8B-405B | 128K | Largest open model |

**Key Innovations:**
- **RMSNorm**: Faster layer normalization
- **SwiGLU activation**: Better than GELU
- **Rotary Position Embeddings (RoPE)**: Efficient position encoding
- **Grouped Query Attention (GQA)**: Efficient inference

In [ ]:
# Model size comparison visualization
import matplotlib.pyplot as plt
import numpy as np

models = {
    'GPT-2': 1.5,
    'GPT-3': 175,
    'LLaMA-7B': 7,
    'LLaMA-70B': 70,
    'LLaMA-405B': 405,
    'GPT-4 (est.)': 1700,  # Estimated MoE total
}

fig, ax = plt.subplots(figsize=(12, 6))

names = list(models.keys())
sizes = list(models.values())
colors = ['#1f77b4', '#1f77b4', '#2ca02c', '#2ca02c', '#2ca02c', '#ff7f0e']

bars = ax.barh(names, sizes, color=colors)
ax.set_xlabel('Parameters (Billions)', fontsize=12)
ax.set_title('LLM Parameter Comparison', fontsize=14)
ax.set_xscale('log')

# Add value labels
for bar, size in zip(bars, sizes):
    ax.text(size * 1.1, bar.get_y() + bar.get_height()/2, 
            f'{size:.0f}B', va='center', fontsize=10)

# Add legend
from matplotlib.patches import Patch
legend_elements = [
    Patch(facecolor='#1f77b4', label='OpenAI GPT'),
    Patch(facecolor='#2ca02c', label='Meta LLaMA'),
    Patch(facecolor='#ff7f0e', label='GPT-4 (MoE est.)'),
]
ax.legend(handles=legend_elements, loc='lower right')

plt.tight_layout()
plt.show()

---
## 6. Training Stages

Modern LLMs go through multiple training stages:

```
┌────────────────────────────────────────────────────────────────────────────┐
│                        LLM Training Pipeline                                │
├────────────────────────────────────────────────────────────────────────────┤
│                                                                             │
│  ┌─────────────────────────────────────────────────────────────────────┐   │
│  │  Stage 1: PRETRAINING                                                │   │
│  │  ─────────────────────                                               │   │
│  │  • Objective: Next token prediction                                  │   │
│  │  • Data: Trillions of tokens (web, books, code)                      │   │
│  │  • Compute: 1000s of GPUs for weeks/months                           │   │
│  │  • Result: Base model with general knowledge                         │   │
│  └─────────────────────────────────────────────────────────────────────┘   │
│                                    │                                        │
│                                    ▼                                        │
│  ┌─────────────────────────────────────────────────────────────────────┐   │
│  │  Stage 2: SUPERVISED FINE-TUNING (SFT)                               │   │
│  │  ─────────────────────────────────────                               │   │
│  │  • Objective: Learn instruction following                            │   │
│  │  • Data: (instruction, response) pairs                               │   │
│  │  • Compute: 10s-100s of GPUs for hours/days                          │   │
│  │  • Result: Instruction-tuned model                                   │   │
│  └─────────────────────────────────────────────────────────────────────┘   │
│                                    │                                        │
│                                    ▼                                        │
│  ┌─────────────────────────────────────────────────────────────────────┐   │
│  │  Stage 3: RLHF (Reinforcement Learning from Human Feedback)          │   │
│  │  ──────────────────────────────────────────────────────              │   │
│  │  • Train reward model on human preferences                           │   │
│  │  • Optimize policy (LLM) using PPO/DPO                               │   │
│  │  • Result: Aligned, helpful, harmless model                          │   │
│  └─────────────────────────────────────────────────────────────────────┘   │
│                                                                             │
└────────────────────────────────────────────────────────────────────────────┘
```

### Stage 1: Pretraining

The foundational training stage where the model learns language patterns.

```
┌─────────────────────────────────────────────────────────────────┐
│                      Pretraining                                 │
├─────────────────────────────────────────────────────────────────┤
│                                                                  │
│   Training Data                        Objective                 │
│   ─────────────                        ─────────                 │
│   • Common Crawl (web)                                           │
│   • Books (Books3, etc.)        "The cat sat on the [?]"        │
│   • Wikipedia                            │                       │
│   • Code (GitHub)                        ▼                       │
│   • Scientific papers            Predict: "mat"                  │
│                                                                  │
│   Loss Function:                                                 │
│   ─────────────                                                  │
│   L = -∑ log P(token_t | token_1, ..., token_{t-1})             │
│       (Cross-entropy loss)                                       │
│                                                                  │
└─────────────────────────────────────────────────────────────────┘
```

In [ ]:
# Simplified pretraining loss calculation
import torch
import torch.nn.functional as F

def compute_lm_loss(logits: torch.Tensor, targets: torch.Tensor) -> torch.Tensor:
    """
    Compute language modeling loss (cross-entropy).
    
    Args:
        logits: Model predictions (batch, seq_len, vocab_size)
        targets: Ground truth token IDs (batch, seq_len)
    """
    # Flatten for cross-entropy
    batch_size, seq_len, vocab_size = logits.shape
    logits_flat = logits.view(-1, vocab_size)
    targets_flat = targets.view(-1)
    
    # Cross-entropy loss
    loss = F.cross_entropy(logits_flat, targets_flat, reduction='mean')
    
    return loss

# Simulate a forward pass
batch_size = 4
seq_len = 128
vocab_size = 50257  # GPT-2 vocabulary size

# Random logits (in practice, from the model)
logits = torch.randn(batch_size, seq_len, vocab_size)
# Random targets (in practice, the next tokens)
targets = torch.randint(0, vocab_size, (batch_size, seq_len))

loss = compute_lm_loss(logits, targets)
perplexity = torch.exp(loss)

print(f"Pretraining Metrics (simulated):")
print(f"  Cross-entropy loss: {loss.item():.4f}")
print(f"  Perplexity: {perplexity.item():.2f}")
print(f"\n  Note: Lower perplexity = better language modeling")
print(f"  Typical pretrained LLM perplexity: 5-20 on held-out data")

### Stage 2: Supervised Fine-Tuning (SFT)

Teaching the model to follow instructions.

```
┌─────────────────────────────────────────────────────────────────┐
│               Supervised Fine-Tuning (SFT)                       │
├─────────────────────────────────────────────────────────────────┤
│                                                                  │
│  Training Data Format:                                           │
│  ─────────────────────                                           │
│                                                                  │
│  ┌─────────────────────────────────────────────────────┐        │
│  │ <s>[INST] Explain quantum computing [/INST]         │        │
│  │ Quantum computing harnesses quantum mechanical      │        │
│  │ phenomena like superposition and entanglement...    │        │
│  │ </s>                                                 │        │
│  └─────────────────────────────────────────────────────┘        │
│                                                                  │
│  Key Aspects:                                                    │
│  • Only compute loss on response tokens (not instruction)       │
│  • Data: 10K-1M high-quality examples                           │
│  • Sources: Human-written, distilled from larger models         │
│                                                                  │
│  Popular SFT Datasets:                                           │
│  • FLAN Collection                                               │
│  • Alpaca (Stanford)                                             │
│  • ShareGPT                                                      │
│  • OpenAssistant                                                 │
│                                                                  │
└─────────────────────────────────────────────────────────────────┘
```

### Stage 3: RLHF (Reinforcement Learning from Human Feedback)

Aligning the model with human preferences.

```
┌────────────────────────────────────────────────────────────────────────────┐
│                              RLHF Pipeline                                  │
├────────────────────────────────────────────────────────────────────────────┤
│                                                                             │
│  Step 1: Collect Human Preferences                                          │
│  ─────────────────────────────────                                          │
│                                                                             │
│    Prompt: "Explain photosynthesis"                                        │
│                                                                             │
│    Response A                      Response B                               │
│    ┌──────────────────┐           ┌──────────────────┐                     │
│    │ Photosynthesis   │           │ Plants eat sun   │                     │
│    │ converts light   │     👍    │ and make food.   │     👎              │
│    │ energy into...   │   BETTER  │                  │    WORSE            │
│    └──────────────────┘           └──────────────────┘                     │
│                                                                             │
│  Step 2: Train Reward Model                                                 │
│  ──────────────────────────                                                 │
│                                                                             │
│    ┌─────────────────────────────────────────────────────┐                 │
│    │  Reward Model (RM)                                  │                 │
│    │  r = RM(prompt, response)                           │                 │
│    │  Loss: -log(σ(r_chosen - r_rejected))               │                 │
│    └─────────────────────────────────────────────────────┘                 │
│                                                                             │
│  Step 3: Optimize Policy with PPO                                           │
│  ────────────────────────────────                                           │
│                                                                             │
│    ┌────────────┐   generate   ┌────────────┐   score   ┌────────────┐    │
│    │   Policy   │───────────►  │  Response  │─────────► │   Reward   │    │
│    │   (LLM)    │              │            │           │   Model    │    │
│    └─────▲──────┘              └────────────┘           └─────┬──────┘    │
│          │                                                    │           │
│          └────────────────── PPO Update ◄─────────────────────┘           │
│                                                                             │
│    Loss = -E[r(x,y)] + β·KL(π_θ || π_ref)                                  │
│           ↑               ↑                                                │
│        Maximize       KL penalty to stay close                             │
│        reward         to original model                                    │
│                                                                             │
└────────────────────────────────────────────────────────────────────────────┘
```

In [ ]:
# Reward model training objective (Bradley-Terry model)
def reward_model_loss(r_chosen: torch.Tensor, r_rejected: torch.Tensor) -> torch.Tensor:
    """
    Compute reward model loss using Bradley-Terry model.
    
    The loss encourages the reward model to assign higher scores
    to preferred responses.
    
    Args:
        r_chosen: Reward for chosen/preferred response
        r_rejected: Reward for rejected response
    """
    # Sigmoid on difference: P(chosen > rejected) = σ(r_chosen - r_rejected)
    return -torch.log(torch.sigmoid(r_chosen - r_rejected)).mean()

# Simulate reward model outputs
batch_size = 16
r_chosen = torch.randn(batch_size) + 1.0    # Higher rewards
r_rejected = torch.randn(batch_size) - 0.5  # Lower rewards

loss = reward_model_loss(r_chosen, r_rejected)

print("Reward Model Training")
print("="*50)
print(f"  Mean reward (chosen):   {r_chosen.mean():.3f}")
print(f"  Mean reward (rejected): {r_rejected.mean():.3f}")
print(f"  Loss: {loss.item():.4f}")
print(f"\n  Note: Loss decreases as the model learns to")
print(f"        distinguish preferred from rejected responses")

### Alternative: Direct Preference Optimization (DPO)

DPO simplifies RLHF by eliminating the need for a separate reward model.

$$\mathcal{L}_{DPO}(\pi_\theta; \pi_{ref}) = -\mathbb{E}\left[\log \sigma\left(\beta \log \frac{\pi_\theta(y_w|x)}{\pi_{ref}(y_w|x)} - \beta \log \frac{\pi_\theta(y_l|x)}{\pi_{ref}(y_l|x)}\right)\right]$$

Where:
- $y_w$: Preferred (winning) response
- $y_l$: Rejected (losing) response
- $\beta$: Temperature parameter
- $\pi_{ref}$: Reference policy (SFT model)

In [ ]:
def dpo_loss(
    policy_chosen_logps: torch.Tensor,
    policy_rejected_logps: torch.Tensor,
    reference_chosen_logps: torch.Tensor,
    reference_rejected_logps: torch.Tensor,
    beta: float = 0.1
) -> torch.Tensor:
    """
    Compute DPO loss.
    
    Args:
        policy_chosen_logps: Log probs of chosen response under policy
        policy_rejected_logps: Log probs of rejected response under policy
        reference_chosen_logps: Log probs of chosen response under reference
        reference_rejected_logps: Log probs of rejected response under reference
        beta: Temperature parameter
    """
    # Compute log ratios
    chosen_logratios = policy_chosen_logps - reference_chosen_logps
    rejected_logratios = policy_rejected_logps - reference_rejected_logps
    
    # DPO loss
    logits = beta * (chosen_logratios - rejected_logratios)
    loss = -F.logsigmoid(logits).mean()
    
    return loss

# Simulate log probabilities
batch_size = 16
policy_chosen = torch.randn(batch_size) - 2.0
policy_rejected = torch.randn(batch_size) - 3.0
ref_chosen = torch.randn(batch_size) - 2.5
ref_rejected = torch.randn(batch_size) - 2.5

loss = dpo_loss(policy_chosen, policy_rejected, ref_chosen, ref_rejected)

print("Direct Preference Optimization (DPO)")
print("="*50)
print(f"  Loss: {loss.item():.4f}")
print(f"\n  Advantages of DPO over RLHF:")
print(f"  • No separate reward model needed")
print(f"  • More stable training (no RL)")
print(f"  • Simpler implementation")

---
## 7. Scaling Laws

**Scaling laws** describe how model performance improves with increased compute, data, and parameters.

### Kaplan Scaling Laws (OpenAI, 2020)

Performance scales as a power law with:

$$L(N) \propto N^{-0.076}$$
$$L(D) \propto D^{-0.095}$$
$$L(C) \propto C^{-0.050}$$

Where:
- $L$: Cross-entropy loss
- $N$: Number of parameters
- $D$: Dataset size (tokens)
- $C$: Compute budget (FLOPs)

### Chinchilla Scaling Laws (DeepMind, 2022)

Key finding: **Models are typically undertrained!**

Optimal ratio: **20 tokens per parameter**

```
┌────────────────────────────────────────────────────────────────────────────┐
│                    Chinchilla Optimal Scaling                               │
├────────────────────────────────────────────────────────────────────────────┤
│                                                                             │
│  For a given compute budget C:                                              │
│                                                                             │
│  ┌─────────────────────────────────────────────────────────────────────┐   │
│  │                                                                      │   │
│  │    N_opt ∝ C^0.5      (Optimal parameters)                          │   │
│  │    D_opt ∝ C^0.5      (Optimal tokens)                              │   │
│  │                                                                      │   │
│  │    N_opt ≈ D_opt / 20                                               │   │
│  │                                                                      │   │
│  └─────────────────────────────────────────────────────────────────────┘   │
│                                                                             │
│  Comparison:                                                                │
│  ───────────                                                                │
│                                                                             │
│  Model        Parameters   Tokens        Tokens/Param   Status             │
│  ─────        ──────────   ──────        ────────────   ──────             │
│  GPT-3        175B         300B          1.7            Undertrained       │
│  Chinchilla   70B          1.4T          20             Optimal            │
│  LLaMA        65B          1.4T          21.5           Optimal            │
│  LLaMA 2      70B          2T            28.6           Well-trained       │
│                                                                             │
│  Both Chinchilla (70B) and GPT-3 (175B) achieve similar performance!       │
│                                                                             │
└────────────────────────────────────────────────────────────────────────────┘
```

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

# Visualize scaling laws
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Left: Loss vs Parameters
ax1 = axes[0]
params = np.logspace(6, 12, 100)  # 1M to 1T parameters
loss_kaplan = 10 * (params / 1e6) ** (-0.076)  # Kaplan scaling

ax1.loglog(params, loss_kaplan, 'b-', linewidth=2)
ax1.set_xlabel('Parameters', fontsize=12)
ax1.set_ylabel('Loss', fontsize=12)
ax1.set_title('Loss vs Model Size (Power Law)', fontsize=14)
ax1.grid(True, alpha=0.3)

# Mark some models
models_params = {
    'GPT-2': 1.5e9,
    'GPT-3': 175e9,
    'LLaMA-70B': 70e9,
}
for name, p in models_params.items():
    loss = 10 * (p / 1e6) ** (-0.076)
    ax1.plot(p, loss, 'ro', markersize=8)
    ax1.annotate(name, (p, loss), xytext=(5, 5), textcoords='offset points')

# Right: Chinchilla optimal scaling
ax2 = axes[1]
compute = np.logspace(19, 25, 100)  # FLOPs
params_optimal = (compute / 6) ** 0.5  # Simplified Chinchilla
tokens_optimal = params_optimal * 20

ax2.loglog(compute, params_optimal, 'b-', linewidth=2, label='Optimal Parameters')
ax2.loglog(compute, tokens_optimal, 'g--', linewidth=2, label='Optimal Tokens')
ax2.set_xlabel('Compute (FLOPs)', fontsize=12)
ax2.set_ylabel('Scale', fontsize=12)
ax2.set_title('Chinchilla Optimal Scaling', fontsize=14)
ax2.legend()
ax2.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print("\nChinchilla Recommendations for Given Compute:")
print("─" * 60)
for flops, name in [(1e21, '10^21 FLOPs'), (1e23, '10^23 FLOPs'), (1e25, '10^25 FLOPs')]:
    opt_params = (flops / 6) ** 0.5
    opt_tokens = opt_params * 20
    print(f"{name}: {opt_params/1e9:.1f}B params, {opt_tokens/1e12:.1f}T tokens")

### Emergent Abilities

Certain capabilities **emerge** only at sufficient scale:

```
┌─────────────────────────────────────────────────────────────────────┐
│                     Emergent Abilities                               │
├─────────────────────────────────────────────────────────────────────┤
│                                                                      │
│  Performance                                                         │
│       │                                          ●─────────●        │
│       │                                      ●                       │
│       │                                   ●                          │
│       │                               ●                              │
│       │  ●────●────●────●────●────●  ← Sharp transition              │
│       │                                                              │
│       └──────────────────────────────────────────────► Model Size   │
│                              ↑                                       │
│                        Threshold scale                               │
│                                                                      │
│  Examples of emergent abilities:                                     │
│  • Chain-of-thought reasoning                                        │
│  • Multi-step arithmetic                                             │
│  • Word unscrambling                                                 │
│  • Code execution prediction                                         │
│  • Theory of mind tasks                                              │
│                                                                      │
│  Debate: Are these truly "emergent" or measurement artifacts?       │
│                                                                      │
└─────────────────────────────────────────────────────────────────────┘
```

---
## Summary

### Key Takeaways

1. **LLMs are next-token predictors** trained on massive text corpora
2. **Transformer architecture** with attention is the foundation
3. **Multi-head attention** allows parallel learning of different patterns
4. **Subword tokenization** (BPE) balances vocabulary and sequence length
5. **Three training stages**: Pretraining → SFT → RLHF/DPO
6. **Scaling laws** guide efficient compute allocation
7. **Emergent abilities** appear at larger scales

### Further Reading

- [Attention Is All You Need](https://arxiv.org/abs/1706.03762) - Original Transformer paper
- [Language Models are Few-Shot Learners](https://arxiv.org/abs/2005.14165) - GPT-3 paper
- [Training Compute-Optimal LLMs](https://arxiv.org/abs/2203.15556) - Chinchilla paper
- [LLaMA: Open Foundation Models](https://arxiv.org/abs/2302.13971) - LLaMA paper
- [Constitutional AI](https://arxiv.org/abs/2212.08073) - Anthropic's approach
- [Direct Preference Optimization](https://arxiv.org/abs/2305.18290) - DPO paper

In [ ]:
# Quick reference: Key hyperparameters for popular LLMs
llm_configs = {
    "GPT-2": {
        "params": "1.5B",
        "layers": 48,
        "d_model": 1600,
        "heads": 25,
        "context": 1024,
        "vocab": 50257,
    },
    "GPT-3 (175B)": {
        "params": "175B",
        "layers": 96,
        "d_model": 12288,
        "heads": 96,
        "context": 2048,
        "vocab": 50257,
    },
    "LLaMA-7B": {
        "params": "7B",
        "layers": 32,
        "d_model": 4096,
        "heads": 32,
        "context": 2048,
        "vocab": 32000,
    },
    "LLaMA-70B": {
        "params": "70B",
        "layers": 80,
        "d_model": 8192,
        "heads": 64,
        "context": 4096,
        "vocab": 32000,
    },
}

print("LLM Architecture Comparison")
print("=" * 80)
print(f"{'Model':<18} {'Params':<10} {'Layers':<8} {'d_model':<10} {'Heads':<8} {'Context':<10} {'Vocab'}")
print("-" * 80)
for model, config in llm_configs.items():
    print(f"{model:<18} {config['params']:<10} {config['layers']:<8} {config['d_model']:<10} {config['heads']:<8} {config['context']:<10} {config['vocab']}")